In [4]:
import os
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import tensorflow as tf
import glob
import fiftyone as fo

In [9]:
# Download airplane images from COCO 2014
# train_dataset = fo.zoo.load_zoo_dataset(
#     "coco-2014",
#     split="train",
#     label_types=["segmentations"],
#     classes=["airplane"],
#     max_samples=800,
#     dataset_dir='./airplane_train'
# )

# val_dataset = fo.zoo.load_zoo_dataset(
#     "coco-2014",
#     split="validation",
#     label_types=["detections"],
#     classes=["airplane"],
#     max_samples=105,
#     dataset_dir='./airplane_val'
# )

Found annotations at 'airplane_val\raw\instances_val2014.json'
96 images found; downloading the remaining 9
 100% |██████████████████████| 9/9 [486.3ms elapsed, 0s remaining, 18.6 images/s]  
Writing annotations for 105 downloaded samples to './airplane_val\validation\labels.json'
Dataset info written to './airplane_val\info.json'
Loading 'coco-2014' split 'validation'
 100% |█████████████████| 105/105 [314.0ms elapsed, 0s remaining, 334.4 samples/s]     
Dataset 'coco-2014-validation-105' created


In [6]:
# Modified from https://colab.research.google.com/github/tensorflow/models/blob/master/research/deeplab/deeplab_demo.ipynb
class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, tarball_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    graph_def = None
    # Extract frozen graph from tar archive.
    tar_file = tarfile.open(tarball_path)
    for tar_info in tar_file.getmembers():
      if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
        file_handle = tar_file.extractfile(tar_info)
        graph_def = tf.compat.v1.GraphDef.FromString(file_handle.read())
        break

    tar_file.close()

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.compat.v1.Session(graph=self.graph)

  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.LANCZOS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map


def create_pascal_label_colormap():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.zeros((256, 3), dtype=int)
  ind = np.arange(256, dtype=int)

  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

  return colormap


def label_to_color_image(label):
  """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')

  colormap = create_pascal_label_colormap()

  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')

  return colormap[label]


# def vis_segmentation(image, seg_map):
#   """Visualizes input image, segmentation map and overlay view."""
#   plt.figure(figsize=(15, 5))
#   grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

#   plt.subplot(grid_spec[0])
#   plt.imshow(image)
#   plt.axis('off')
#   plt.title('input image')

#   plt.subplot(grid_spec[1])
#   seg_image = label_to_color_image(seg_map).astype(np.uint8)
#   plt.imshow(seg_image)
#   plt.axis('off')
#   plt.title('segmentation map')

#   plt.subplot(grid_spec[2])
#   plt.imshow(image)
#   plt.imshow(seg_image, alpha=0.7)
#   plt.axis('off')
#   plt.title('segmentation overlay')

#   unique_labels = np.unique(seg_map)
#   ax = plt.subplot(grid_spec[3])
#   plt.imshow(
#       FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
#   ax.yaxis.tick_right()
#   plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
#   plt.xticks([], [])
#   ax.tick_params(width=0.0)
#   plt.grid('off')
#   plt.show()


LABEL_NAMES = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

In [ ]:
MODEL_NAME = 'xception_coco_voctrainval'  # @param ['mobilenetv2_coco_voctrainaug', 'mobilenetv2_coco_voctrainval', 'xception_coco_voctrainaug', 'xception_coco_voctrainval']

_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
_MODEL_URLS = {
    'mobilenetv2_coco_voctrainaug':
        'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
    'mobilenetv2_coco_voctrainval':
        'deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz',
    'xception_coco_voctrainaug':
        'deeplabv3_pascal_train_aug_2018_01_04.tar.gz',
    'xception_coco_voctrainval':
        'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
}
_TARBALL_NAME = 'deeplab_model.tar.gz'

model_dir = tempfile.mkdtemp()
tf.io.gfile.makedirs(model_dir)

download_path = os.path.join(model_dir, _TARBALL_NAME)
print('downloading model, this might take a while...')
urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME],
                   download_path)
print('download completed! loading DeepLab model...')

MODEL = DeepLabModel(download_path)
print('model loaded successfully!')

In [10]:
def make_img_background_green(is_train=True):
  """Inferences DeepLab model and visualizes result."""
  if (is_train):
    images = glob.glob("airplane_train/train/data/*.jpg")
  else:
    images = glob.glob("airplane_val/validation/data/*.jpg")
  print('running deeplab on images...')
  for i in range(len(images)):
    original_im = Image.open(images[i])
    resized_im, seg_map = MODEL.run(original_im)
    im_as_array, map_as_array = np.array(resized_im), np.array(seg_map)
    im_as_array[map_as_array != 1] = (0, 100, 0)
    image_with_green_bg = Image.fromarray(im_as_array)
    if (is_train):
      image_with_green_bg.save('./input-with-green-screen-background/image%s.jpg' % i)
    else:
      image_with_green_bg.save('./val-with-green-screen-background/image%s.jpg' % i)

# make_img_background_green()
# make_img_background_green(is_train=False)

running deeplab on images...


In [11]:
import warnings; warnings.filterwarnings('ignore');

import sys
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.pyplot import imread

import skimage
from skimage import img_as_ubyte
from skimage.transform import rescale
from skimage.transform import rotate
from skimage import io
from skimage import filters
from skimage import exposure
from skimage.morphology import disk
from skimage.filters import rank
import re

In [12]:
# Credit https://github.com/RaulSanchezVazquez/chroma-key-composition
def remove_green(img):
    """
    Docstring:
        Remove green-ish background from image given a threshold.

    Parameters
    ----------
    img : numpy.array containing 4 channel image (RGBa).
    """
    
    norm_factor = 255

    """
    Obtain the ratio of the green/red/blue
    channels based on the max-bright of 
    the pixel.
    """
    
    red_ratio = img[:, :, 0] / norm_factor
    green_ratio = img[:, :, 1] / norm_factor
    blue_ratio = img[:, :, 2] / norm_factor

    """
    Darker pixels would be around 0.
    In order to ommit removing dark pixels we
    sum .2 to make small negative numbers to be
    above 0.
    """
    
    red_vs_green = (red_ratio - green_ratio) + .2
    blue_vs_green = (blue_ratio - green_ratio) + .2

    """
    Now pixels below 0. value would have a
    high probability to be background green
    pixels.
    """
    red_vs_green[red_vs_green < 0] = 0
    blue_vs_green[blue_vs_green < 0] = 0

    """
    Combine the red(blue) vs green ratios to
    set an alpha layer with valid alpha-values.
    """
    alpha = (red_vs_green + blue_vs_green) * 255
    alpha[alpha > 50] = 255

    """
    Set the alpha layer
    """
    img[:, :, 3] = alpha

    return img

In [13]:
def transform_img(img, scale=None, angle=None):
    #Perform scaling
    if not(scale is None):
        img = rescale(
            img, 
            scale,
            channel_axis=2)
    
    #Perform rotation
    if not(angle is None):
        img = rotate(
            img, 
            angle,
            resize=True, 
            mode='edge')
        
    return img

# Credit https://github.com/RaulSanchezVazquez/chroma-key-composition
def blend(bg, img, coord=(0, 0)):
    """
    Docstring:
        Blends two images, one as background and one on top, the image
        on top must be an image with alpha layer in order to perform
        organic blendings.

    Parameters
    ----------
    bg : Background image
    img : numpy.array, 4 channel image (RBGa)
    coord : coordinates of the blending position of img in bg.
    scale : Scale operation to perform on img [-Inf, Inf]
    angle : Rotation angle in degrees in img.
    
    Return
    ----------
    np.array : blended composition
    """
    img = img_as_ubyte(img)
    
    (x_size, y_size, _) = img.shape

    (y_ini, x_ini) = coord
    x_end = x_ini + x_size
    y_end = y_ini + y_size

    bg_crop = bg[
        x_ini:x_end,
        y_ini:y_end, 
        :]

    pixel_preserve = (img[:, :, -1] > 10)
    bg_crop[pixel_preserve] = img[pixel_preserve]

    bg[x_ini:x_end, y_ini:y_end, :] = bg_crop

    return bg

In [14]:
def sigmoid(x):
    """
    params
    ----------
    x : np.array with x values of a sigmoid
    Sigmoid function.
    
    return
    ----------
    np.array : y-sigmoid values of x
    """
    return 1 / (1 + np.exp(-x))

# Credit https://github.com/RaulSanchezVazquez/chroma-key-composition
def channel_adjust(img, curve, channel):
    """
    Adjust channel.
    
    Thanks to:
        http://www.practicepython.org/blog/2016/12/20/instagram-filters-python.html
    params
    ------
    single_img_channel : 2-D array corresponding to a single channel image
    curve : Curve to which correct the pixels brightness
    
    returns
    ------
    np.array : The adjusted channel
    """
    regular_inntervals = np.linspace(0, 1, len(curve))
    
    adjustment_flat = np.interp(
        img[:, :, channel].ravel(), 
        regular_inntervals,
        curve)
    shape = img[:, :, channel].shape
    
    adjustment = adjustment_flat.reshape(shape)
    img[:, :, channel] = adjustment
    
    return

In [15]:
# Credit https://github.com/RaulSanchezVazquez/chroma-key-composition
def merge_alpha(img, bg):
    """
    
    Thanks to:
        https://stackoverflow.com/questions/25182421/overlay-two-numpy-arrays-treating-fourth-plane-as-alpha-level
    """
    src_rgb = img[..., :3]
    src_a = img[..., 3]

    dst_rgb = bg[..., :3]
    dst_a = bg[..., 3]

    out_a = src_a + dst_a * (1.0 - src_a)
    out_rgb = (
        src_rgb*src_a[..., None] + dst_rgb*dst_a[..., None]*(1.0 - src_a[..., None])) / out_a[..., None]

    out = np.zeros_like(bg)
    out[..., :3] = out_rgb
    out[..., 3] = out_a
    
    return out

In [21]:
imgs_with_green_bg = sorted(glob.glob("input-with-green-screen-background/*.jpg"))
# imgs_with_green_bg = sorted(glob.glob("val-with-green-screen-background/*.jpg"))
# backgrounds = glob.glob("backgrounds/*.jpg")
backgrounds = glob.glob("diff-backgrounds/*.jpg")
save_path = './diff-outputs/image%s_with_bg_%s.jpg'
for inp_img in imgs_with_green_bg:
    for j in range(len(backgrounds)):
        i = re.search(r'\d+', inp_img).group()
        img = Image.open(inp_img).convert('RGBA')
        bg_img = Image.open(backgrounds[j]).convert('RGBA')
        
        img_as_array = np.array(img).copy()
        bg_as_array = np.array(bg_img).copy()
        remove_green(img_as_array)

        scale = np.random.uniform(0.5, 1.5)
        angle = np.random.randint(-180, 181)

        img_as_array = transform_img(img_as_array, scale=scale, angle=angle)
        ws, hs = img_as_array.shape[1], img_as_array.shape[0]
        bgw, bgh = bg_as_array.shape[1], bg_as_array.shape[0]
        x_coord = np.random.randint(0, bgw-ws)
        y_coord = np.random.randint(0, bgh-hs)

        blended = blend(
            bg_as_array, img_as_array,
            coord=(x_coord, y_coord)
        )
        #Make adjust curves
        red_adjust_curve = sigmoid(np.arange(-3, 5, .35))
        green_adjust_curve = sigmoid(np.arange(-5, 5, .45))
        blue_adjust_curve = sigmoid(np.arange(-2.3, 2.3, .2))

        img_effect = skimage.img_as_float(
            blended.copy())

        channel_adjust(
            img_effect, 
            red_adjust_curve, 
            channel=0)

        channel_adjust(
            img_effect, 
            green_adjust_curve, 
            channel=1)

        channel_adjust(
            img_effect, 
            blue_adjust_curve, 
            channel=2)

        blurred = filters.gaussian(
            img_effect.copy(),
            sigma=3,
            channel_axis=2)

        shape_0 = blurred.shape[0]
        shape_1 = blurred.shape[1]
        blur_gradient = (1 - np.arange(0, 1, 1 / shape_1))

        blur_gradient = blur_gradient.reshape(
            1, -1
        ).repeat(
            shape_0, axis=0
        ).reshape(shape_0, shape_1)

        blurred[:, :, 3] = 1 - blur_gradient

        final = merge_alpha(blurred, img_effect)
        final_img = Image.fromarray((final * 255).astype(np.uint8)).convert('RGB')
        # final_img.save('./outputs/image%s_with_bg_%s.jpg' % (i, j))
        # final_img.save('./val-outputs/image%s_with_bg_%s.jpg' % (i, j))
        final_img.save(save_path % (i, j))